<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

In [2]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [44]:
# текст запроса
query_3_1 = '''
SELECT count(*)
FROM public.vacancies
'''

In [45]:
# результат запроса
pd.read_sql_query(query_3_1, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_15116\1469115235.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [46]:
# текст запроса
query_3_2 = '''
SELECT count(*)
FROM public.employers
'''

In [47]:
# результат запроса
pd.read_sql_query(query_3_2, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_15116\4057088587.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [48]:
# текст запроса
query_3_3 = '''
SELECT count(*)
FROM public.areas
'''

In [49]:
# результат запроса
pd.read_sql_query(query_3_3, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_15116\4147122770.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [50]:
# текст запроса
query_3_4 = '''
SELECT count(*)
FROM public.industries
'''

In [51]:
# результат запроса
pd.read_sql_query(query_3_4, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_15116\2627729322.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_4, connection)


,count
0,294


***

**Вывод:**

  + По предварительному анализу видно, что судя по количеству регионов, вакансии размещены и за прелелами России в том числе. 
  + Количество вакансий превышает количество работодателей незначительно, всего в два раза, а значит в таблице будет присутствовать большое количество мелких работодателей.
---

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [52]:
# Соединим таблицу с вакансиями и таблицу с регионами по ключу id_area и сгруппируем по названию локации,
# посчитаем количество вхождений для каждой локации.

query_4_1 = '''
select 
    a.name area,
    count(*) cnt
from public.vacancies v
    join public.areas a on v.area_id = a.id
group by 1
order by 2 desc
'''

In [53]:
# результат запроса
pd.read_sql_query(query_4_1, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_15116\4222419522.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_1, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [6]:
# Отфильтруем таблицу, записав в условие, что хотя бы верхняя или нижняя граница зарплаты указана.
 
query_4_2 = '''
select 
    count(*)
from public.vacancies 
where salary_from is not null or salary_to is not null
'''

In [7]:
# результат запроса
pd.read_sql_query(query_4_2, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_14832\614600247.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_2, connection)


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [26]:
# Выведем два столбца: с нижней и верхней границей зарплаты и посчитаем среднее по этим столбцам.

query_4_3 = '''
select 
    avg(salary_from)::int avg_salary_from,
    avg(salary_to)::int avg_salary_to
from public.vacancies 
'''

In [27]:
# результат запроса
pd.read_sql_query(query_4_3, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_8892\3534961570.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_3, connection)


,avg_salary_from,avg_salary_to
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [28]:
# Сгруппируем таблицу сначала по типу рабочего графика, затем по типу трудоустройства, посчитаем 
# количество вхождений для каждого сочетания.

query_4_4 = '''
select
    schedule,
    employment,
    count(*)
from public.vacancies 
group by 1, 2
order by 3 desc
'''

In [44]:
# результат запроса
pd.read_sql_query(query_4_4, connection).head()

C:\Users\User\AppData\Local\Temp\ipykernel_8892\1870616927.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_4, connection).head()


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [31]:
# Сгруппируем таблицу по признаку требуемый опыт работы и посчитаем количество вхождений для каждого 
# значения, затем отсортируем в требуемом порядке.

query_4_5 = '''
select
    experience,
    count(*)
from public.vacancies 
group by 1
order by 2
'''

In [32]:
# результат запроса
pd.read_sql_query(query_4_5, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_8892\3190483664.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_5, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

**Вывод:**

  + Среди городов по количеству вакансий безоговорочным лидером является Москва, опережая с большим отрывом второй Санкт-Петербург, остальные города распределены более равномерно. 
  + Только у половины вакансий заполнено хотя бы одно из двух полей с зарплатой. В то же время среди тех вакансий в которых указана ЗП, в среднем она превышает среднюю ЗП по стране. На мой взгляд тут 2 причины: первая - это то, что работодатели умышленно не указывают низкие зарплаты, чтобы не отпугивать соискателей, а вторая - то, что некоторые работодатели предпочитают предлагать ЗП уже после собеседования, когда будет понятен уровень соискателя.
  + Подавляющее количество вакансий предусматривает полную занятость, выкансии с работой офлайн преобладает над удаленной работой. Это говорит о том, что большее количество работы невозможно выполнять удаленно.
  + Исходя из данных по количеству вакансий в разрезе опыта работы можно сделать вывод, что в основном компании ищут работников с малым опытом через интернет ресурсы. Предполагаю, что на вакансии требующие большого опыта компании рассматривают кандидатов из своего штата, повышая их в должности, либо предлагая должность конкретным людям.
---

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [33]:
# Объединим таблицу с вакансиями и таблицу с работодателями по ключу employer_id, затем сгруппируем 
# получившуюся таблицу по названию работодателя и посчитаем количество вхождений для каждого работодателя.
# Отсортируем результат в порядке убывания и пронумеруем каждую строку, потом выведем 1 и 5 строки.

query_5_1 = '''
with cte as
(
select
    e.name employer_name,
    row_number() over (order by count(e.name) desc) rn
from public.vacancies v 
    join public.employers e on v.employer_id = e.id
group by 1
)

select employer_name 
from cte 
where rn in (1, 5)
'''

In [35]:
# результат запроса
pd.read_sql_query(query_5_1, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_8892\2425258503.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_1, connection).head()


,employer_name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [8]:
# 1. К таблице с локациями присоединим таблицу с вакансиями по id локации и таблицу с работодателями по 
# также по id локации с помощью метода left join. Таким образом мы получим таблицу в которой будут 
# пропуски в тех местах где не нашлось соответствующих значений для таблицы с локациями.
# 2. Отфильтруем получившуюся таблицу, оставив в ней те строки, в которых нет вакансий
# 3. Получившуюся таблицу сгруппируем по названию локации и посчитаем количество вакансий и работодателей 
# для каждой локации. 
# 4. Отсортируем в порядке убывания количества работодателей.
 
query_5_2 = '''
select 
    a.name area,
    count(v.id) cnt_vacancion,
    count(e.id) cnt_employers
from public.areas a
    left join public.vacancies v on a.id = v.area_id
    left join public.employers e on a.id = e.area
where v.id is null
group by 1
order by 3 desc
'''

In [12]:
# результат запроса
pd.read_sql_query(query_5_2, connection).head()

C:\Users\User\AppData\Local\Temp\ipykernel_14832\2701199214.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_2, connection).head()


,area,cnt_vacancion,cnt_employers
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [17]:
# 1. Объединим таблицу с вакансиями с таблицей с работодателями используя join, таким образом мы 
# получим таблицу где будут находиться названия только тех работодателей, у которых есть вакансии.
# 2. Сгруппируем по названию региона 
# 3. Посчиаем количество уникальных вхождений 
# 4. Отсортируем по убыванию количества.

query_5_3 = '''
select 
    e.name employer,
    count(distinct v.area_id) cnt
from public.employers e
    join public.vacancies v on e.id = v.employer_id
group by 1 
order by 2 desc 
'''

In [18]:
# результат запроса
pd.read_sql_query(query_5_3, connection).head()

C:\Users\User\AppData\Local\Temp\ipykernel_14832\2485320004.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_3, connection).head()


,employer,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [19]:
# 1. Для объединения таблиц с работодателями и сферами деятельности воспользуемся вспомогательной 
# таблицей employers_industries. Присоеденим с помощью right join таблицу с работодателями, а с помощью 
# left join таблицу со сферами деятельности. Таким образом получится результирующая таблица в которой будут 
# все работодатели, даже те, у кого нет соответствия из таблицы со сферами деятельности
# 2. Отфильтруем таблицу и оставим те строки, в которых не указана сфера деятельности.
# 3. Посчитаем количество этих строк.

query_5_4 = '''
select 
    count(*)
from public.employers_industries ei 
    right join employers e on ei.employer_id = e.id
    left join public.industries i on ei.industry_id = i.id 
where i.id is null
'''

In [20]:
# результат запроса
pd.read_sql_query(query_5_4, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_14832\1644714913.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_4, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [13]:
# 1. Используя вспомогательную таблицу employers_industries объединим таблицы с работодателями и сферами 
# деятельности только теперь уже при помощи join и получим только те компании, у которых указаны сферы 
# деятельности.
# 2. Сгруппируем таблицу по названию компании и посчитаем количество сфер деятельности для каждой компании
# 3. Отфильтруем таблицу, оставив только те компании, у которых указано 4 сферы деятельности
# 4. Отсортируем таблицу в алфавитном порядке

query_5_5 = '''
select 
    distinct e.name name_company,
    count(i.id) cnt_industries
from public.employers_industries ei 
    join employers e on ei.employer_id = e.id
    join public.industries i on ei.industry_id = i.id 
group by 1 
having count(i.id) = 4
order by 1
'''

In [16]:
# результат запроса
pd.read_sql_query(query_5_5, connection).head()

C:\Users\User\AppData\Local\Temp\ipykernel_9572\2261468984.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_5, connection).head()


,name_company,cnt_industries
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [17]:
# 1. Объединим таблицы с работодателями и сферами деятельности так же, как и в предыдущем задании 
# 2. Отфильтруем, оставив те строки, в которых сфера деятельности 'Разработка программного обеспечения'
# 3. Посчитаем количество получившихся строк.

query_5_6 = '''
select 
    count(*)
from public.employers_industries ei 
    join employers e on ei.employer_id = e.id
    join public.industries i on ei.industry_id = i.id 
where i.name = 'Разработка программного обеспечения'
'''

In [20]:
# результат запроса
pd.read_sql_query(query_5_6, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_9572\2878008418.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_6, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [4]:
# Скопируем ссылку на страницу с сайта
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

df = pd.read_html(url)[1] # Считаем нужную нам таблицу в переменную df
cities = tuple(df['Город']) # Выделим колонку с названиями городов и создадим кортеж.
cities # Выведем результат

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [17]:
# 1. К таблице с вакансиями присоединим таблицу с работодателями и таблицу с регионами, чтобы 
# получить названия компаний и регионов.
# 2. Отфильтруем данную таблицу так, чтобы остались строки, где компания 'Яндекс' и город относится к 
# списку городов-миллионников
# 3. Соединим эту таблицу саму с собой, только в одной части сгруппируем таблицу по названию города и 
# посчитаем количество строк для каждого города, а во второй части посчитаем количество строк во всей 
# таблице
# 4. Отсортируем получившуюся таблицу по возрастанию количества.

query_5_7 = f''' 
with cte as
(
select 
    a.name city,
    count(*) cnt
from public.vacancies v 
    join public.employers e on v.employer_id = e.id
    join public.areas a on v.area_id = a.id
where e.name = 'Яндекс' and a.name in {cities}
group by 1     
)

select city, cnt 
from cte
union all
select 'Total', sum(cnt)::int
from cte
order by 2
'''

In [18]:
# результат запроса
pd.read_sql_query(query_5_7, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_4172\1216511710.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_7, connection)


,city,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

**Вывод:**

  + На первом месте по количеству вакансий и по количеству присутствия в разных регионах с большим отрывом находится Яндекс, что говорит о том что он является самой быстрорастущей компанией России
  + В топе по присутствию в разных регионах есть 3 работодателя, которые занимают высокие места, являясь довольно мелкими: Спецремонт, Поляков Денис Иванович, ООО ЕФИН. Это происходит потому, что они просто размещают одни и те же вакансии в разных регионах.
  + Примерно треть работодателей не указывает сферу деятельности, предполагаю что это маленькие компании и ИП
  + Крупные компании указывают большое количество сфер деятельности. 
---

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [27]:
# Отфильтруем таблицу при помощи метода ilike для игнорирования регистра и посчитаем количество
# получившихся строк.
 
query_6_1 = '''
select  
    count(*)
from public.vacancies 
where name ilike '%data%' or name ilike '%данн%'
'''

In [28]:
# результат запроса
pd.read_sql_query(query_6_1, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_18864\2603769340.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [29]:
# Отфильтруем таблицу используя фильтры указанные в задании. Чтобы исключить вакансии по html из нашей 
# выборки, добавим условие not ilike '%HTML%', чтобы убрать html в любом регистре. Чтобы не включать в
# выборку строки с TeamLead включаем в выборку 'ML' только в верхнем регистре. Посчитаем количество строк.

query_6_2 = '''
select  
    count(*)
from vacancies 
where 
    (name ilike '%data scientist%' or name ilike '%data science%' 
    or name ilike '%исследователь данных%' or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%' or name like '%ML%' and name not ilike '%HTML%')
    and (name ilike '%junior%' or experience = 'Нет опыта' or employment = 'Стажировка')
'''

In [30]:
# результат запроса
pd.read_sql_query(query_6_2, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_18864\2428688723.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [35]:
# Возьмем за основу фильтры из прошлого задания, уберем ограничения связанные с junior - специалистом и
# добавим ограничения связанные с ключевыми словами в столбце 'key_skills'. Посчитаем количество строк.

query_6_3 = '''
select  
    count(*)
from vacancies 
where 
    (name ilike '%data scientist%' or name ilike '%data science%' 
    or name ilike '%исследователь данных%' or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%' or name like '%ML%' and name not ilike '%HTML%')
    and (key_skills ilike '%SQL%' or key_skills ilike '%postgres%')
'''

In [36]:
# результат запроса
pd.read_sql_query(query_6_3, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_15116\3902879435.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [37]:
# Возьмем запрос из прошлого задания, и поменяем фильтр для колонки 'key_skills' на ключевое слово 'Python'.

query_6_4 = '''
select  
    count(*)
from vacancies 
where 
    (name ilike '%data scientist%' or name ilike '%data science%' 
    or name ilike '%исследователь данных%' or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%' or name like '%ML%' and name not ilike '%HTML%')
    and (key_skills ilike '%Python%')
'''

In [38]:
# результат запроса
pd.read_sql_query(query_6_4, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_15116\2548700122.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [40]:
# 1. Отфильтруем таблицу используя фильтры из предыдущего задания для того чтобы остались только вакаесии 
# связаннные с Data Science
# 2. Навыки разделены табуляцией, поэтому количество навыков будет всегда на 1 больше чем количество 
# табуляций. Поэтому посчитаем изначальную длину строки, затем уберем табуляцию и вновь посчитаем длину,
# найдем разницу этих значений и прибавим к ней 1
# 3. Вычислим среднее значение по данному столбцу и округлим до двух знаков после запятой.

query_6_5 = '''
select  
    round(avg(length(key_skills) - length(replace(key_skills, CHR(9), ''))+1), 2)
    from vacancies
where 
    (name ilike '%data scientist%' or name ilike '%data science%' 
    or name ilike '%исследователь данных%' or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%' or name like '%ML%' and name not ilike '%HTML%')
'''

In [41]:
# результат запроса
pd.read_sql_query(query_6_5, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_15116\3854041181.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_5, connection)


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [42]:
# 1. Отфильтруем таблицу, оставив только те строки, которые относятся к Data Science и в которых заполнено 
# хотя бы одно поле с зарплатой.
# 2. Сгруппируем полученную таблицу по признаку опыта и посчитаем среднюю зарплату для каждого значения.
# 3. В столбце со средней зарплатой используем метод coalesce, в который перым аргументом передадим рассчет 
# средней зарплаты для двух значений, вторым аргументом salary_from, а третьим salary_to.

query_6_6 = '''
select  
   experience,
   round(avg(coalesce((salary_from + salary_to)/2, salary_from, salary_to))) mean_salary
   from vacancies
where 
    (name ilike '%data scientist%' or name ilike '%data science%' 
    or name ilike '%исследователь данных%' or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%' or name like '%ML%' and name not ilike '%HTML%')
    and (salary_from is not null or salary_to is not null)
group by 1
'''


In [43]:
# результат запроса
pd.read_sql_query(query_6_6, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_15116\1796641676.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_6, connection)


,experience,mean_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

**Вывод:**

  + Исходя из полученных данных видно, что вакансий связанных с работой с данными относительно немного, примерно 2,6% от общего числа, среди этих вакансий очень невелика часть вакансий, подходящая для junior специалистов, примерно 4%. Это говорит о том, что компании стремятся искать специалистов с опытом работы.
  + В среднем зарплаты в работе с данными превышают средние зарплаты по всем отраслям. Отмечу достаточно сильный рост зарплаты с ростом опыта в данной профессии.
---

# Юнит 7. Дополнительные исследования

1. Выведите топ среди городов-миллионников России по средней заработной плате. В рассчет брать только те записи, в которых заполнено хотя бы одно поле с заработной платой. Если заполнено обе записи, то найти среднее между ними, если только какая-то одна запись, то считать ее средней. Результат округлить до целого.

In [44]:
# 1. Объединим таблицу с регионами и таблицу с вакансиями методом join
# 2. Отфильтруем таблицу, оставив только записи с городами-миллионниками (используем котеж cities) у которых
# заполнено хотя бы одно поле с зарплатой
# 3. Сгруппируем таблицу по названию города и посчитаем среднюю зарплату.
# 4. Отсортируем в порядке убывания 

query_7_1 = f'''
select  
   a.name city,
   round(avg(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)))::int mean_salary
from public.vacancies v 
   join public.areas a on v.area_id = a.id
where (v.salary_from is not null or v.salary_to is not null) and a.name in {cities}
group by 1
order by 2 desc
'''

In [45]:
# результат запроса
pd.read_sql_query(query_7_1, connection)

C:\Users\User\AppData\Local\Temp\ipykernel_14832\2677872968.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_7_1, connection)


,city,mean_salary
0,Москва,134140
1,Новосибирск,107777
2,Санкт-Петербург,104410
3,Нижний Новгород,96059
4,Пермь,95985
5,Ростов-на-Дону,94065
6,Краснодар,92328
7,Казань,92207
8,Екатеринбург,91896
9,Омск,86393


2. Посчитайте распределение вакансий связанных с Data Science по регионам. Отсортируйте полученные данные в порядке убывания количества вакансий.

In [46]:
# 1. Объединим таблицу с вакансиями и таблицу с регионами методом join
# 2. Отфильтруем таблицу, оставив только те строки, которые относятся к Data Science 
# 3. Сгруппируем полученную таблицу по названию рениона и посчитаем количество вакансий для каждого региона.
# 4. Отсортируем в порядке убывания вакансий

query_7_2 = '''
select  
    a.name region,
    count(*)
from public.vacancies v 
    join public.areas a on v.area_id = a.id
where 
    v.name ilike '%data scientist%' or v.name ilike '%data science%' 
    or v.name ilike '%исследователь данных%' or v.name ilike '%machine learning%'
    or v.name ilike '%машинн%обучен%' or v.name like '%ML%' and v.name not ilike '%HTML%'
group by 1
order by 2 desc 
'''

In [47]:
# результат запроса
pd.read_sql_query(query_7_2, connection).head(10)

C:\Users\User\AppData\Local\Temp\ipykernel_14832\1975143852.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_7_2, connection).head(10)


,region,count
0,Москва,217
1,Санкт-Петербург,64
2,Новосибирск,23
3,Нижний Новгород,20
4,Казань,16
5,Алматы,16
6,Минск,9
7,Томск,8
8,Екатеринбург,7
9,Краснодар,6


3. Посчитайте среднюю зарплату по регионам для вакансий связанных с Data Science, а также выведите количество этих вакансий в этом регионе. Округлите полученные результаты до целого и отсортируйте в порядке убывания. 

  В рассчет брать только те записи, в которых заполнено хотя бы одно поле с заработной платой. Если заполнено обе записи, то найти среднее между ними, если только какая-то одна запись, то считать ее средней.

In [48]:
# 1. Объединим таблицу с вакансиями и таблицу с регионами методом join
# 2. Отфильтруем таблицу, оставив только те строки, которые относятся к Data Science 
# 3. Сгруппируем полученную таблицу по названию рениона и посчитаем количество вакансий для каждого региона.
# 4. Отсортируем в порядке убывания вакансий

query_7_3 = '''
select
    a.name region,
    count(*) cnt_vacancies,
    round(avg(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to))) mean_salary
from public.vacancies v 
    join public.areas a on v.area_id = a.id
where 
    (v.name ilike '%data scientist%' or v.name ilike '%data science%' 
    or v.name ilike '%исследователь данных%' or v.name ilike '%machine learning%'
    or v.name ilike '%машинн%обучен%' or v.name like '%ML%' and v.name not ilike '%HTML%')
    and (v.salary_from is not null or v.salary_to is not null)
group by 1 
order by 3 desc  
'''

In [49]:
# результат запроса
pd.read_sql_query(query_7_3, connection).head(10)

C:\Users\User\AppData\Local\Temp\ipykernel_14832\3484300568.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_7_3, connection).head(10)


,region,cnt_vacancies,mean_salary
0,Кипр,1,300000.0
1,Армения,3,268863.0
2,Черногория,1,233794.0
3,Турция,1,233794.0
4,Сербия,1,233794.0
5,Москва,27,207630.0
6,Белгород,1,200000.0
7,Санкт-Петербург,7,173335.0
8,Новосибирск,3,166333.0
9,Рязань,3,136667.0


In [ ]:
# не забываем закрыть соединение после окончания работы
connection.close()

# Общий вывод по проекту

**Вывод:**

  + Исходя из анализа видно, что количество вакансий напрямую зависит от населения города. Например Москва которая является лидером по количеству вакансий с большим отрывом от остальных примерно с таким же отрывом опережает всех и по количеству населения 
  + Также замечу, что количество населения влияет и на показатель средней зарплаты по городу, т.е. чем крупнее город, тем выше в среднем по нему зарплата.
  + Отмечу, что количество вакансий наибольшее в категории работников с опытом работы но при этом небольшим. Больше всего вакансий для работников с опытом 1-3 года, затем с опытом 3-6 лет, затем без опыта и только потом с опытом больше 6 лет. На мой взгляд такое распределение происходит от того, ято опытных специалистов выращивают сами компании, а если требуется все таки найти опытного специалиста, то это делается в основном не через подобные ресурсы, а путем прямого предложения конкретному кандидату.
  + Что касается работодателей, то тут можно отметить безоговорочное лидерство Яндекс, а также в топе присутствует большое количество компаний связанных с IT-технологиями, что говорит о том, что это растущие и развивающиеся компании.

**Дополнительно:**

Я провел 3 дополнительных исследования:
  - вывел топ городов-миллионников России по заработной плате;
  - вывел количество вакансий, связанных с данными, распределенное по регионам;
  - посчитал среднюю зарплату по вакансиям, связанным с данными по регионам.

##### Для более детального анализа можно взять аналогичную таблицу за более ранние годы (если она есть) и понаблюдать как меняется рынок труда с течением времени. 
---